In [9]:
from torchvision import transforms
import torchvision
import torch

transform = transforms.Compose([
    transforms.ToTensor(),
])

data = torchvision.datasets.FashionMNIST('path', download = True,transform=transform)
data_loader = torch.utils.data.DataLoader(data,
 batch_size=16,
 shuffle=True)


Podział na zbiór treningowy i tesowy


In [10]:
from torch.utils.data import random_split, DataLoader



total_size = len(data)
train_size = int(0.8 * total_size)  # 80% danych uczących
test_size = total_size - train_size  # 20% danych testowych

train_dataset, test_dataset = random_split(data, [train_size, test_size])
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

Definicja modelu

In [11]:
from torch import nn


class FashionMNISTModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(FashionMNISTModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = x.flatten(start_dim=1)  # Spłaszcz tensor wejściowy
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x
   

Trening

In [12]:
from torch import optim

input_size = 28 * 28  # Rozmiar obrazu FashionMNIST
hidden_size1 = 128
hidden_size2 = 64
output_size = 10  # 10 klas w FashionMNIST
model = FashionMNISTModel(input_size, hidden_size1, hidden_size2, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trening modelu
num_epochs = 5
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader):
        # Przygotuj dane
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Wsteczna propagacja i aktualizacja wag
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Wyświetl postęp
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(data_loader)}], Loss: {loss.item():.4f}')

print("Trening zakończony pomyślnie!")

Epoch [1/5], Step [100/3750], Loss: 0.5784
Epoch [1/5], Step [200/3750], Loss: 0.9642
Epoch [1/5], Step [300/3750], Loss: 0.4115
Epoch [1/5], Step [400/3750], Loss: 0.4095
Epoch [1/5], Step [500/3750], Loss: 0.3238
Epoch [1/5], Step [600/3750], Loss: 0.4906
Epoch [1/5], Step [700/3750], Loss: 0.3457
Epoch [1/5], Step [800/3750], Loss: 0.6338
Epoch [1/5], Step [900/3750], Loss: 0.2652
Epoch [1/5], Step [1000/3750], Loss: 0.6269
Epoch [1/5], Step [1100/3750], Loss: 0.3903
Epoch [1/5], Step [1200/3750], Loss: 0.4097
Epoch [1/5], Step [1300/3750], Loss: 0.6353
Epoch [1/5], Step [1400/3750], Loss: 0.2457
Epoch [1/5], Step [1500/3750], Loss: 0.7461
Epoch [1/5], Step [1600/3750], Loss: 0.7202
Epoch [1/5], Step [1700/3750], Loss: 0.3783
Epoch [1/5], Step [1800/3750], Loss: 0.2316
Epoch [1/5], Step [1900/3750], Loss: 0.4582
Epoch [1/5], Step [2000/3750], Loss: 0.3465
Epoch [1/5], Step [2100/3750], Loss: 0.7960
Epoch [1/5], Step [2200/3750], Loss: 0.3918
Epoch [1/5], Step [2300/3750], Loss: 0.31

Test modelu na zbiorze testowym

In [13]:
model.eval()  # Ustaw model w tryb ewaluacji (wyłącza dropout i batch normalization)

correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Dokładność modelu na zbiorze testowym: {accuracy * 100:.2f}%')


Dokładność modelu na zbiorze testowym: 90.58%
